## The Walks of Nathan Lowell
A look at Nate's path to the back gate.

The journey begins [#tommw](https://www.nathanlowell.com/tommw/)

In [1]:
from io import BytesIO
from PIL import Image
import json

from bs4 import BeautifulSoup
import requests

import photo as pto

[#tommw](https://www.nathanlowell.com/tommw/) is a cross channel social media campaign architeched and executed by science fiction author Nathan Lowell. 

There are 4 planks to Nathan's campain:
1. A blog hosted at https://www.nathanlowell.com/tommw/
2. A photo stream hosted on [FlickR](https://www.flickr.com/photos/nlowell/)
3. Podcast feed via [RSS](http://www.nathanlowell.com/tommw/feed/)
4. Posts to twitter with the #tommw hastag on [@nlowell on twitter](https://twitter.com/nlowell)

In [2]:
# Load the archive file
try:
    with open('archive.json', 'r') as file:
        archive= json.loads(file.read())
except:
    archive = {}
    print('no archived file')
        
# Scrape the blog for links to the other media components
blog_page = requests.get("https://www.nathanlowell.com/tommw/")
soup = BeautifulSoup(blog_page.text, 'html.parser')

# Find a flickr link
flickr = soup.find_all(href=True, attrs={'data-flickr-embed':'true'})
flickr_link = flickr[0]['href']

no archived file


In [3]:
def process_photo_stream_page(stream):
    
    for photo in stream:
        image_name = photo.split('/')[-1]
        photo_id = image_name.split('_')[0]

        if archive.get(photo_id):
            # skip photos already recorded
            continue

        thumb_name = photo_id + '.jpg'

        info = pto.get_original_photo(photo_id)

        if info:
            info['photo_id'] = photo_id
            info['thumb'] = thumb_name
            p_url = info.get('link')
            camera, timestamp = pto.get_exif(p_url)
            info['camera'] = camera
            info['timestamp'] = timestamp
            archive[photo_id] = info
            print(photo_id + ' added')

        else:
            print('no #tommow tag found')
            
    return True

In [4]:
stream = pto.get_photostream(flickr_link)
process_photo_stream_page(stream)

45538428704 added
32390140018 added
31253793227 added
46180203671 added
32292571558 added
31207048157 added
46117344171 added
44213546090 added
45104952125 added
32116506418 added
44077082110 added
30922848627 added
30906651997 added
45747471562 added
45716342292 added
45700451932 added
45008315434 added
43897749190 added
31825102818 added
30740491377 added
30725706957 added
45531832902 added
44474828355 added
44405595765 added


True

In [6]:
with open('archive.json', 'w') as file:
    file.write(json.dumps(archive, indent=4))

Archive File Structure

``` python
{'id':'123456789','info':{'timestamp':'2018-12-10 10:21', ...}}
```
